# Recursos Web
![](https://assets.grepsr.com/grepsr-homepage-v2/wp-content/uploads/2013/12/web.png)

Hasta ahora hemos recurrido a muchas fuentes de información: entrada de usuarios, archivos de diferentes formatos, bases de datos y tablas web. Pero el gran reporitorio de información actualmente es la World Wide Web. Actualmente todo esta en el web y tenemos dos formas de sacarle provecho: utilizando servicios de consultas abiertos basados en APIs, o recorriendo las páginas web desde el código HTML para extraer lo que requerimos utilizando una técnica llamada webscrapping.

Luis A. Muñoz (2022)

# API REST
Igual que una interfaz de usuario permite la interacción y comunicación entre un software y una persona, una API (acrónimo de Application Programming Interface) facilita la relación entre dos aplicaciones para el intercambio de mensajes o datos. Un conjunto de funciones y procedimientos que ofrece una biblioteca para que otro software la utilice como capa de abstracción, un espacio de acceso e intercambio de información adicional en la parte superior. Así una se sirve de la información de la otra sin dejar de ser independientes. 

REST es un estilo de arquitectura de software que se utiliza para describir cualquier interfaz entre diferentes sistemas que utilice HTTP para comunicarse. Este término significa REpresentational State Transfer (transferencia de estado representacional), lo que quiere decir que entre dos llamadas cualquiera, el servicio no guarda los datos. Por ejemplo, podemos autenticar a un usuario con su email y contraseña en una llamada, pero la siguiente que hagamos ya se habrá olvidado de la anterior petición de autenticación.

Una API REST es un backend capaz de contestar a las llamadas a una serie de URLs en formato JSON. Es utilizado por muchas empresas gracias a su versatilidad y eficiencia.

La principal ventaja de las API REST es que podemos desarrollar una API en el backend y utilizarla en cualquier dispositivo, ahorrando así mucho tiempo de desarrollo. En el caso de Twitter, la misma API se consume desde Android, iOS y el navegador web.

<img src="https://phpenthusiast.com/theme/assets/images/blog/what_is_rest_api.png" alt="drawing" width="500"/>

## API REST con request
Para utilizar las facilidades de un API REST, es necesario contar con el URI (Uniform Resource Identifier) del recurso y un método de obtener las respuestas ante las peticiones de recursos o acciones sobre el servicio.

![](https://avaldes.com/wp-content/uploads/2017/08/REST_URL_structure.png)

Para manipular los recursos, HTTP nos dota de los siguientes métodos con los cuales debemos operar:

* GET: Para consultar y leer recursos
* POST: Para crear recursos
* PUT: Para editar recursos
* DELETE: Para eliminar recursos.
* PATCH: Para editar partes concretas de un recurso.

Los métodos de uso más común son GET y POST. Por ejemplo, se puede utilizar GET para saber las ultimas actualizaciones de una cuenta de Twitter, o utilizar POST para enviar un Twitter bajo una cuenta de usuario.

In [1]:
import requests
import json
import logging

### Public API REST
Existen API REST públicas, es decír de libre consulta para obtener información sobre algunos recursos disponibles de dominio público. Por ejemplo, se puede obtener información libre de la Estación Espacial Internacional (IIS) en el proyecto de fuente abierta [Open Notify](http://open-notify.org/). Aquí podemos obtener algunos datos sobre la Estación Espacial y sus tripulantes.

In [6]:
URL = "http://api.open-notify.org/iss-now.json"
r = requests.get(URL)

In [7]:
r.status_code == requests.codes.ok

True

In [8]:
URL = "http://api.open-notify.org/iss-now.json"
r = requests.get(URL)

if r.status_code == requests.codes.ok:
    data = r.json()
    
    print(json.dumps(data, indent=4))
else:
    logging.error(f"Response: {r.status_code}")

{
    "iss_position": {
        "longitude": "-17.1307",
        "latitude": "38.9157"
    },
    "timestamp": 1685652430,
    "message": "success"
}


In [9]:
from datetime import datetime

print(f"Date & Time: {datetime.fromtimestamp(int(data['timestamp']))}")
print(f"Lat: {data['iss_position']['latitude']}, Lon:{data['iss_position']['longitude']}")

Date & Time: 2023-06-01 15:47:10
Lat: 38.9157, Lon:-17.1307


## App: ISS Tracking

In [2]:
from PIL import Image, ImageTk
from datetime import datetime
import tkinter as tk
import requests

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("ISS Tracker")
        self.resizable(0, 0)
        self.config(bg='black')
        
        FONT = ("Terminal", 16, "bold")
        
        img = Image.open("satellite.png")
        img = img.resize((160, 100))
        img = ImageTk.PhotoImage(img)
        
        frm = tk.Frame(self, bg='black')
        frm.pack(padx=10, pady=10)
        
        self.lblLat = tk.Label(frm, text="Latitude : ", font=FONT, fg="#49DC0A", bg='black')
        self.lblLon = tk.Label(frm, text="Longitude:", font=FONT, fg="#49DC0A", bg='black')
        self.lblIcon = tk.Label(frm, image=img, bg='black')
        self.lblIcon.image = img
        
        self.lblLat.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblLon.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblIcon.grid(row=0, column=1, rowspan=2, padx=5, pady=5, sticky=tk.W)
        
        self.update_iss()
        
        
    def update_iss(self):
        URL = "http://api.open-notify.org/iss-now.json"
        r = requests.get(URL)

        if r.status_code == requests.codes.ok:
            data = r.json()
            self.lblLat.config(text=f"Latitude : {data['iss_position']['latitude']}°")
            self.lblLon.config(text=f"Longitude: {data['iss_position']['longitude']}°")
        
        self.after(1000, self.update_iss)

        
App().mainloop()

### API REST con parametros
La librería `request` tiene como principal utilidad la conformación de un URI a partir de un URL; esto quiere decir, agregarle los parametros requeridos por un GET para obtener la información necesaria.

Por ejemplo, para obtener los momentos en los que se puede tener visibilidad de la Estacion Espacial, se puede hacer una consulta sobre el servicio, agregandole la locación de observación.

In [11]:
# Fuente: http://www.7timer.info/doc.php
URL = "https://www.7timer.info/bin/civillight.php"

payload = {'lat': -12.1186707, 
           'lon': -77.0219015,
           'lan': 'en', 
           'unit': 'metric', 
           'output': 'json', 
           'tzshift': 0}

r = requests.get(URL, params=payload)

if r.status_code == requests.codes.ok:
    data = r.json()
    print(json.dumps(data, indent=4))
else:
    logging.error(f"Response: {r.status_code}")

{
    "product": "civillight",
    "init": "2023060112",
    "dataseries": [
        {
            "date": 20230601,
            "weather": "clear",
            "temp2m": {
                "max": 20,
                "min": 20
            },
            "wind10m_max": 3
        },
        {
            "date": 20230602,
            "weather": "pcloudy",
            "temp2m": {
                "max": 20,
                "min": 18
            },
            "wind10m_max": 3
        },
        {
            "date": 20230603,
            "weather": "mcloudy",
            "temp2m": {
                "max": 20,
                "min": 18
            },
            "wind10m_max": 3
        },
        {
            "date": 20230604,
            "weather": "ishower",
            "temp2m": {
                "max": 20,
                "min": 18
            },
            "wind10m_max": 3
        },
        {
            "date": 20230605,
            "weather": "ishower",
            "temp2m": {
   

## App: Weather App

In [ ]:
import tkinter as tk
import requests
from PIL import Image, ImageTk

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Weather App")
        self.resizable(0, 0)
        
        self.wind_data = {1: 'Below 0.3m/s (calm)',
                          2: '0.3-3.4m/s (light)',
                          3: '3.4-8.0m/s (moderate)',
                          4: '8.0-10.8m/s (fresh)', 
                          5: '10.8-17.2m/s (strong)', 
                          6: '17.2-24.5m/s (gale)', 
                          7: '24.5-32.6m/s (storm)', 
                          8: 'Over 32.6m/s (hurricane)'}

        img = Image.open(".\\icons\\empty.png")
        img = ImageTk.PhotoImage(img)
        
        self.var_lat = tk.StringVar()
        self.var_lon = tk.StringVar()
        self.var_lat.set('-12.0023281')
        self.var_lon.set('-76.823428')
        
        frm = tk.Frame(self)
        frm.pack(padx=10, pady=10)
        
        frm1 = tk.LabelFrame(self, text="Input Data")
        frm2 = tk.LabelFrame(self, text="Weather Data")
        frm1.pack(padx=10, pady=10, ipadx=10, ipady=10, anchor=tk.W)
        frm2.pack(padx=10, pady=10, expand=True, fill=tk.X, anchor=tk.W)
        
        #---------------------------------- frm1 ----------------------------------
        lblLat = tk.Label(frm1, text="Latitude:")
        lblLon = tk.Label(frm1, text="Longitude:")
        entLat = tk.Entry(frm1, textvariable=self.var_lat)
        entLon = tk.Entry(frm1, textvariable=self.var_lon)
        btnGetWeather = tk.Button(frm1, text="Get Weather", width=12, command=self.get_weather)
        btnClear = tk.Button(frm1, text="Clear Entrys", width=12, command=self.clear_entrys)
        
        lblLat.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        lblLon.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        entLat.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        entLon.grid(row=1, column=1, padx=5, pady=5, sticky=tk.W)
        btnGetWeather.grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        btnClear.grid(row=1, column=2, padx=5, pady=5, sticky=tk.W)
        
        #---------------------------------- frm2 ----------------------------------
        self.lblWeather = tk.Label(frm2, text="Weather: ")
        self.lblTemp = tk.Label(frm2, text="Temp Min | Max: ")
        self.lblWind = tk.Label(frm2, text="Wind: ")
        self.lblIcon = tk.Label(frm2, image=img)
        
        self.lblWeather.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblTemp.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblWind.grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblIcon.grid(row=0, column=1, rowspan=2, padx=5, pady=5, sticky=tk.W)
        
        
    def get_weather(self):
        URL = "https://www.7timer.info/bin/civillight.php"
        payload = {'lat': self.var_lat.get(), 
                   'lon': self.var_lon.get(),
                   'lan': 'en',
                   'unit': 'metric',
                   'output': 'json' , 
                   'tzshift': 0}

        r = requests.get(URL, params=payload)

        if r.status_code == requests.codes.ok:
            data = r.json()
            
            img = Image.open(f".\\icons\\{data['dataseries'][0]['weather']}.png")
            img = ImageTk.PhotoImage(img)
            self.lblIcon.config(image=img)
            self.lblIcon.image = img
            
            self.lblWeather.config(text=f"Weather: {data['dataseries'][0]['weather']}")
            self.lblTemp.config(text=f"Temp Min | Max: {data['dataseries'][0]['temp2m']['min']}°C | {data['dataseries'][0]['temp2m']['max']}°C")
            self.lblWind.config(text=f"Wind: {self.wind_data.get(data['dataseries'][0]['wind10m_max'])}")
            
    
    def clear_entrys(self):
        self.var_lat.set('0.000000')
        self.var_lon.set('0.000000')
        
        img = Image.open(f".\\icons\\empty.png")
        img = ImageTk.PhotoImage(img)
        self.lblIcon.config(image=img)
        self.lblIcon.image = img

        self.lblWeather.config(text="Weather: ")
        self.lblTemp.config(text="Temp Min | Max: ")
        self.lblWind.config(text="Wind: ")



App().mainloop()

### Public Key API REST
Existen API REST que requieren de un tipo de autenticación por parte del usuario simple, en forma de una código alfanumérico llamado llave (ya sea privada, de consumo, etc.) que debe ser generada como parte de un proceso de creación de cuenta de desarrollador. Por ejemplo, el API REST de [MapQuest](https://developer.mapquest.com/) permite obtener muchos servicios geográficos, pero requiere el uso de una llave de autenticación, por lo que será necesario abrir una cuenta y utilizar la llave para utilizar los recursos.

### TIP: Secretos y Variables de Ambiente
Esta llave de autenticación se debe de enviar cada vez que se hace una consulta sobre el API, y es una buena práctica mantenerla secreta y lejos del código. Hay algunas formas de mantener las cosas secretas en Python, y una de ellas es utilizar las *variables de ambiente* y a librería `doyenv`. Será necesario instalar:

    pip install -U python-dotenv
    
Una vez hecho esto, debemos crear un archivo `.env` con un editor de texto en la misma ruta de trabajo con la siguiente informacion (la llave mostrada es ficticia, tendrá que colocar la suya):

    MAPQUEST_KEY = 8sHGh5hHGH69JHJjkiusljsijss
    
Para utilizar la llave utilizaremos el siguiente código:

In [ ]:
!pip install -U python-dotenv

In [15]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

API_KEY = os.environ.get("MAPQUEST_KEY")
#print(API_KEY)

Ahora con la clave en un lugar seguro, podemos llmar al API REST Geocoding. Este permite obtener las coordenadas geográficas a partir de una dirección.

In [16]:
from dotenv import load_dotenv, find_dotenv
import requests
import json
import os

load_dotenv(find_dotenv())
API_KEY = os.environ.get("MAPQUEST_KEY")

URL = "https://www.mapquestapi.com/geocoding/v1/address"
payload = {'key': API_KEY, 
           'location': 'Av. La Marina 2810, San Miguel, Lima'}
r = requests.get(URL, params=payload)
print(f"GET {r.url}")

if r.status_code == requests.codes.ok:
    data = r.json()
    
    # Guardar en un archivo (temp.json)
    with open("temp.json", mode='w') as file:
        json.dump(data, file)
else:
    print(r.status_code)

GET https://www.mapquestapi.com/geocoding/v1/address?key=VSHBijCtLwMxAAzMeITIGuOcdnGPc5O9&location=Av.+La+Marina+2810%2C+San+Miguel%2C+Lima


## App: Weather App con información de ciudad

In [17]:
import os
import requests
import tkinter as tk
import tkinter.ttk as ttk
from PIL import Image, ImageTk

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Weather App")
        self.resizable(0, 0)
        
        frm = tk.Frame(self)
        frm.pack(padx=10, pady=10)
        
        frm1 = tk.LabelFrame(self, text="Input Data")
        frm2 = tk.LabelFrame(self, text="Output")
        frm1.pack(padx=10, pady=10, ipadx=10, ipady=10)
        frm2.pack(padx=10, pady=10, expand=True, fill=tk.X)
        
        # ---------------------------------- frm1 ------------------------------
        lblCiudad = tk.Label(frm1, text="Ciudad:")
        self.cboCiudad = ttk.Combobox(frm1, state='readonly', 
                                 values=sorted(['Lima, Perú', 
                                                'Santiago, Chile', 
                                                'Bogotá, Colombia', 
                                                'Quito, Ecuador', 
                                                'Brasilia, Brasil',
                                                'Buenos Aires, Argentina',
                                                'Caracas, Venezuela',
                                                'La Paz, Bolivia',
                                                'Asunción, Paraguay',
                                                'Montevideo, Uruguay']))
        btnGetWeather = tk.Button(frm1, text="Get Weather", width=12, command=self.get_weather)
        btnClear = tk.Button(frm1, text="Clear", width=12, command=self.clear)
        
        lblCiudad.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.cboCiudad.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        btnGetWeather.grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        btnClear.grid(row=0, column=3, padx=5, pady=5, sticky=tk.W)
        
        # ---------------------------------- frm2 ------------------------------
        self.lblWeather = tk.Label(frm2, text="Weather: ")
        self.lblTemp = tk.Label(frm2, text="Temp Max | Temp Min: ")
        self.lblWind = tk.Label(frm2, text="Wind: ")
        self.lblIcon = tk.Label(frm2)
        
        self.lblWeather.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblTemp.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblWind.grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblIcon.grid(row=0, column=1, rowspan=2, padx=5, pady=5, sticky=tk.W)
        
        
    def get_weather(self):
        # Consulta en API MapQuest para obtener Lat y Lon
        API_KEY = os.environ.get("MAPQUEST_KEY")
        URL = "http://open.mapquestapi.com/geocoding/v1/address"
        payload = {'key': API_KEY, 
                   'location': self.cboCiudad.get()}
        r = requests.get(URL, params=payload)

        if r.status_code == requests.codes.ok:
            data = r.json()
            
            # Consulta en API 7timer para obtener la informacion del clima
            URL = "https://www.7timer.info/bin/civillight.php"

            payload = {'lat': data['results'][0]['locations'][0]['latLng']['lat'], 
                       'lon': data['results'][0]['locations'][0]['latLng']['lng'],
                       'lan': 'en', 
                       'unit': 'metric', 
                       'output': 'json', 
                       'tzshift': 0}

            wind_data = {1: 'Below 0.3m/s (calm)',
                         2: '0.3-3.4m/s (light)',
                         3: '3.4-8.0m/s (moderate)',
                         4: '8.0-10.8m/s (fresh)', 
                         5: '10.8-17.2m/s (strong)', 
                         6: '17.2-24.5m/s (gale)', 
                         7: '24.5-32.6m/s (storm)', 
                         8: 'Over 32.6m/s (hurricane)'}

            r = requests.get(URL, params=payload)

            if r.status_code == requests.codes.ok:
                data = r.json()

                img_png = Image.open(f".\\icons\\{data['dataseries'][0]['weather']}.png")
                img = ImageTk.PhotoImage(img_png)

                self.lblWeather.config(text=f"Weather: {data['dataseries'][0]['weather']}")
                self.lblTemp.config(text=f"Temp Min | Temp Max: {data['dataseries'][0]['temp2m']['min']}°C | {data['dataseries'][0]['temp2m']['max']}°C")
                self.lblWind.config(text=f"Wind: {wind_data[data['dataseries'][0]['wind10m_max']]}")

                self.lblIcon.config(image=img)
                self.lblIcon.image = img
            else:
                print(f"ERROR: Response: {r.status_code}")

            
    def clear(self):
        img_png = Image.open(f".\\icons\\empty.png")
        img = ImageTk.PhotoImage(img_png)

        self.lblWeather.config(text=f"Weather: ")
        self.lblTemp.config(text=f"Temp Min | Temp Max: ")
        self.lblWind.config(text=f"Wind: ")

        self.lblIcon.config(image=img)
        self.lblIcon.image = img
        
        
App().mainloop()

## App: Weather App con informacion de ciudad

In [ ]:
import os
import requests
import tkinter as tk
import tkinter.ttk as ttk
from PIL import Image, ImageTk
from dotenv import load_dotenv, find_dotenv

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Weather App")
        self.resizable(0, 0)
        
        load_dotenv(find_dotenv())
        self.API_KEY = os.environ.get("MAPQUEST_KEY")
 
        self.wind_data = {1: 'Below 0.3m/s (calm)',
                          2: '0.3-3.4m/s (light)',
                          3: '3.4-8.0m/s (moderate)',
                          4: '8.0-10.8m/s (fresh)', 
                          5: '10.8-17.2m/s (strong)', 
                          6: '17.2-24.5m/s (gale)', 
                          7: '24.5-32.6m/s (storm)', 
                          8: 'Over 32.6m/s (hurricane)'}

        frm = tk.Frame(self)
        frm.pack(padx=10, pady=10)
        
        frm1 = tk.LabelFrame(self, text="Input Data")
        frm2 = tk.LabelFrame(self, text="Weather Data")
        frm1.pack(padx=10, pady=10, ipadx=10, ipady=10)
        frm2.pack(padx=10, pady=10, expand=True, fill=tk.X)
        
        # ---------------------------------- frm1 ------------------------------
        lblCiudad = tk.Label(frm1, text="Ciudad:")
        self.cboCiudad = ttk.Combobox(frm1, state='readonly', 
                                 values=sorted(['Lima, Perú', 
                                                'Santiago, Chile', 
                                                'Bogotá, Colombia', 
                                                'Quito, Ecuador', 
                                                'Brasilia, Brasil',
                                                'Buenos Aires, Argentina',
                                                'Caracas, Venezuela',
                                                'La Paz, Bolivia',
                                                'Asunción, Paraguay',
                                                'Montevideo, Uruguay', 
                                                'Managua, Philippines']))
        btnGetWeather = tk.Button(frm1, text="Get Weather", width=12, command=self.get_weather)
        btnClear = tk.Button(frm1, text="Clear", width=12, command=self.clear)
        
        lblCiudad.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.cboCiudad.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        btnGetWeather.grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        btnClear.grid(row=0, column=3, padx=5, pady=5, sticky=tk.W)
        
        
        # ---------------------------------- frm2 ------------------------------
        self.lblWeather = tk.Label(frm2, text="Weather: ")
        self.lblTemp = tk.Label(frm2, text="Temp Max | Temp Min: ")
        self.lblWind = tk.Label(frm2, text="Wind: ")
        self.lblIcon = tk.Label(frm2)
        
        self.lblWeather.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblTemp.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblWind.grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblIcon.grid(row=0, column=1, rowspan=2, padx=5, pady=5, sticky=tk.W)
         
                                                
    def get_weather(self):
        # Se obtiene lat, lon de la ccboCiudadad
        URL = "https://www.mapquestapi.com/geocoding/v1/address"
        payload = {'key': self.API_KEY, 
                   'location': self.cboCiudad.get()}

        r = requests.get(URL, params=payload)

        if r.status_code == requests.codes.ok:
            data = r.json()
            lat = data['results'][0]['locations'][0]['latLng']['lat']
            lon = data['results'][0]['locations'][0]['latLng']['lng']
            
            # Se obtiene la informacion del clima de lat, lon
            URL = "https://www.7timer.info/bin/civillight.php"
            payload = {'lat': lat, 
                       'lon': lon,
                       'lan': 'en',
                       'unit': 'metric',
                       'output': 'json' , 
                       'tzshift': 0}

            r = requests.get(URL, params=payload)

            if r.status_code == requests.codes.ok:
                data = r.json()

                img = Image.open(f".\\icons\\{data['dataseries'][0]['weather']}.png")
                img = ImageTk.PhotoImage(img)
                self.lblIcon.config(image=img)
                self.lblIcon.image = img

                self.lblWeather.config(text=f"Weather: {data['dataseries'][0]['weather']}")
                self.lblTemp.config(text=f"Temp Min | Max: {data['dataseries'][0]['temp2m']['min']}°C | {data['dataseries'][0]['temp2m']['max']}°C")
                self.lblWind.config(text=f"Wind: {self.wind_data.get(data['dataseries'][0]['wind10m_max'])}")


        
    def clear(self):
        img_png = Image.open(f".\\icons\\empty.png")
        img = ImageTk.PhotoImage(img_png)

        self.lblWeather.config(text=f"Weather: ")
        self.lblTemp.config(text=f"Temp Min | Temp Max: ")
        self.lblWind.config(text=f"Wind: ")

        self.lblIcon.config(image=img)
        self.lblIcon.image = img
        
        
App().mainloop()

### Bonus Track: Widget Notebook

In [ ]:
import os
import requests
import tkinter as tk
import tkinter.ttk as ttk
from PIL import Image, ImageTk
from dotenv import load_dotenv, find_dotenv
from datetime import datetime, timedelta

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Weather App")
        self.resizable(0, 0)
        
        load_dotenv(find_dotenv())
        self.API_KEY = os.environ.get("MAPQUEST_KEY")
 
        self.wind_data = {1: 'Below 0.3m/s (calm)',
                          2: '0.3-3.4m/s (light)',
                          3: '3.4-8.0m/s (moderate)',
                          4: '8.0-10.8m/s (fresh)', 
                          5: '10.8-17.2m/s (strong)', 
                          6: '17.2-24.5m/s (gale)', 
                          7: '24.5-32.6m/s (storm)', 
                          8: 'Over 32.6m/s (hurricane)'}

        frm = tk.Frame(self)
        frm.pack(padx=10, pady=10)
        
        frmUp = tk.LabelFrame(frm, text="Input Data")
        frmUp.pack(padx=10, pady=10, ipadx=10, ipady=10)
        
        notebook = ttk.Notebook(frm)
        notebook.pack(padx=10, pady=10, expand=True, fill=tk.X)
        
        dates = [f"{datetime.now() + timedelta(days=idx): %d/%m/%Y}" for idx in range(6)]
        frames = [tk.Frame(notebook) for _ in range(6)]
                
        for frame, date in zip(frames, dates):
            frame.pack(padx=10, pady=10)
            notebook.add(frame, text=date)
        
        # ---------------------------------- frmUp ------------------------------
        lblCiudad = tk.Label(frmUp, text="Ciudad:")
        self.cboCiudad = ttk.Combobox(frmUp, state='readonly', width=24, 
                                 values=sorted(['Lima, Perú', 
                                                'Santiago, Chile', 
                                                'Bogotá, Colombia', 
                                                'Quito, Ecuador', 
                                                'Brasilia, Brasil',
                                                'Buenos Aires, Argentina',
                                                'Caracas, Venezuela',
                                                'La Paz, Bolivia',
                                                'Asunción, Paraguay',
                                                'Montevideo, Uruguay']))
        btnGetWeather = tk.Button(frmUp, text="Get Weather", width=12, command=self.get_weather)
        btnClear = tk.Button(frmUp, text="Clear", width=12, command=self.clear)
        
        lblCiudad.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.cboCiudad.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        btnGetWeather.grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        btnClear.grid(row=0, column=3, padx=5, pady=5, sticky=tk.W)
                
        # ------------------- frames Notebook -----------------------------        
        self.widgets = []
        for idx, frm in enumerate(frames):
            self.widgets.append([tk.Label(frm, text="Weather: "), 
                        tk.Label(frm, text="Temp Max | Temp Min: "), 
                        tk.Label(frm, text="Wind: "), 
                        tk.Label(frm)])           
  
            self.widgets[idx][0].grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
            self.widgets[idx][1].grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
            self.widgets[idx][2].grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
            self.widgets[idx][3].grid(row=0, column=1, rowspan=2, padx=5, pady=5, sticky=tk.W)
  

    @staticmethod
    def get_geolocation(API_KEY, city):
        URL = "https://www.mapquestapi.com/geocoding/v1/address"
        payload = {'key': API_KEY, 
                   'location': city}

        r = requests.get(URL, params=payload)

        if r.status_code == requests.codes.ok:
            data = r.json()
            lat = data['results'][0]['locations'][0]['latLng']['lat']
            lon = data['results'][0]['locations'][0]['latLng']['lng']
            return lat, lon

    
    def get_weather(self):
        # Se obtiene lat, lon de la cboCiudadad
        if self.cboCiudad.get():
            lat, lon = App.get_geolocation(self.API_KEY, self.cboCiudad.get())

            # Se obtiene la informacion del clima de lat, lon
            URL = "https://www.7timer.info/bin/civillight.php"
            payload = {'lat': lat, 'lon': lon,
                       'lan': 'en', 'unit': 'metric',
                       'output': 'json', 'tzshift': 0}

            r = requests.get(URL, params=payload)

            if r.status_code == requests.codes.ok:
                data = r.json()

                for idx, widgets in enumerate(self.widgets):
                    img = Image.open(f".\\icons\\{data['dataseries'][idx]['weather']}.png")
                    img = ImageTk.PhotoImage(img)
                    widgets[3].config(image=img)
                    widgets[3].image = img

                    widgets[0].config(text=f"Weather: {data['dataseries'][idx]['weather']}")
                    widgets[1].config(text=f"Temp Min | Max: {data['dataseries'][idx]['temp2m']['min']}°C | {data['dataseries'][0]['temp2m']['max']}°C")
                    widgets[2].config(text=f"Wind: {self.wind_data.get(data['dataseries'][idx]['wind10m_max'])}")

        
    def clear(self):
        for idx, widgets in enumerate(self.widgets):
            img = Image.open(f".\\icons\\empty.png")
            img = ImageTk.PhotoImage(img)
            widgets[3].config(image=img)
            widgets[3].image = img

            widgets[0].config(text=f"Weather: ")
            widgets[1].config(text=f"Temp Min | Max: ")
            widgets[2].config(text=f"Wind: ")

        
        
App().mainloop()

### API REST Directions

In [18]:
URL = "http://www.mapquestapi.com/directions/v2/route"
payload = {'key': API_KEY, 
           'from': 'Av. Primavera 2310, Santiago de Surco, Lima', 
           'to': 'La Marina 2810, San Miguel, Lima', 
           'unit': 'k', 
           'locale': 'es_MX'}
r = requests.get(URL, params=payload)
print(f"GET {r.url}")

if r.status_code == requests.codes.ok:
    data = r.json()
    
    # Guardar en un archivo
    with open("temp.json", mode='w') as file:
        json.dump(data, file)
else:
    print(r.status_code)

GET http://www.mapquestapi.com/directions/v2/route?key=VSHBijCtLwMxAAzMeITIGuOcdnGPc5O9&from=Av.+Primavera+2310%2C+Santiago+de+Surco%2C+Lima&to=La+Marina+2810%2C+San+Miguel%2C+Lima&unit=k&locale=es_MX


## App: Ruta de un punto a otro

In [ ]:
import os
import requests
import tkinter as tk
import tkinter.ttk as ttk
from tkinter.scrolledtext import ScrolledText
from dotenv import load_dotenv, find_dotenv
from datetime import datetime

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Route App")
        self.resizable(0, 0)
        
        load_dotenv(find_dotenv())
        self.API_KEY = os.environ.get("MAPQUEST_KEY")
 
        frm = tk.Frame(self)
        frm.pack(padx=10, pady=10)
        
        self.lblDesde = tk.Label(frm, text="Desde:")
        self.lblHacia = tk.Label(frm, text="Hacia:")
        self.entDesde = tk.Entry(frm, width=50)
        self.entHacia = tk.Entry(frm, width=50)
        self.btnRoute = tk.Button(frm, text="Route Map", width=12, command=self.route)
        self.text_area = ScrolledText(frm, wrap=tk.WORD)
        
        self.lblDesde.grid(row=0, column=0, padx=5, pady=5, sticky=tk.E)
        self.lblHacia.grid(row=1, column=0, padx=5, pady=5, sticky=tk.E)
        self.entDesde.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        self.entHacia.grid(row=1, column=1, padx=5, pady=5, sticky=tk.W)
        self.btnRoute.grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        self.text_area.grid(row=2, column=0, columnspan=3, padx=10, pady=10)
        
    def route(self):
        if self.entDesde.get() and self.entHacia.get():
            URL = "http://www.mapquestapi.com/directions/v2/route"
            payload = {'key': self.API_KEY, 
                      'from': self.entDesde.get(), 
                      'to': self.entHacia.get(), 
                      'unit': 'k',
                      'locale': 'es_MX'}
            
            r = requests.get(URL, params=payload)
            
            if r.status_code == requests.codes.ok:
                data = r.json()
                
                for idx, item in enumerate(data['route']['legs'][0]['maneuvers'], start=1):
                    self.text_area.insert(f"{idx}.0", f"[{idx}]. {item['narrative']}\n")

    
App().mainloop()

## Clientes API a servicios (SDK)
Existen clientes que pueden ser importados como modulos en Python que se encargan de gestionar todas las llamadas a los diferentes servicios, sin tener que recurrir a llamadas directas. Estos se conocen como SDK (Software Development Kit). Algunos ejemplos de estos son:

* geopy
* folium

In [ ]:
!pip install geopy

In [19]:
# pip install geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")
location = geolocator.geocode("UPC Campus San Miguel")

In [20]:
location.address

'UPC - Campus San Miguel, 2810, Avenida La Marina, Virgen de Fatima, San Miguel, Lima, Lima Metropolitana, Lima, 15087, Perú'

In [21]:
location.raw

{'place_id': 201947939,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 418674556,
 'boundingbox': ['-12.0770408', '-12.0758134', '-77.0942605', '-77.0929534'],
 'lat': '-12.076417849999999',
 'lon': '-77.09360681318614',
 'display_name': 'UPC - Campus San Miguel, 2810, Avenida La Marina, Virgen de Fatima, San Miguel, Lima, Lima Metropolitana, Lima, 15087, Perú',
 'class': 'amenity',
 'type': 'university',
 'importance': 0.40001,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/education_university.p.20.png'}

In [ ]:
!pip install folium

In [22]:
# pip install folium
import folium

In [23]:
m = folium.Map(location=[location.latitude, location.longitude],zoom_start=18)
folium.Marker(location=[location.latitude, location.longitude], popup='UPC San Miguel', icon=folium.Icon()).add_to(m)
m

## App: Generador de rutas en un mapa

In [ ]:
!pip install tkintermapview

In [ ]:
# Ref: https://github.com/TomSchimansky/TkinterMapView
import os
import tkinter as tk
import tkintermapview
import requests
from dotenv import load_dotenv, find_dotenv

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Route Map")
        self.resizable(0, 0)
        
        load_dotenv(find_dotenv())
        self.API_KEY = os.environ.get("MAPQUEST_KEY")
 
        frm = tk.Frame(self)
        frm.pack(padx=10, pady=10)
        
        self.lblDesde = tk.Label(frm, text="Desde:")
        self.lblHacia = tk.Label(frm, text="Hacia:")
        self.entDesde = tk.Entry(frm, width=50)
        self.entHacia = tk.Entry(frm, width=50)
        self.btnRoute = tk.Button(frm, text="Route Map", width=12, command=self.route)
        self.map = tkintermapview.TkinterMapView(frm, width=800, height=600, corner_radius=0)
        
        self.lblDesde.grid(row=0, column=0, padx=5, pady=5, sticky=tk.E)
        self.lblHacia.grid(row=1, column=0, padx=5, pady=5, sticky=tk.E)
        self.entDesde.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        self.entHacia.grid(row=1, column=1, padx=5, pady=5, sticky=tk.W)
        self.btnRoute.grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        self.map.grid(row=2, column=0, columnspan=3, padx=10, pady=10)
        
        self.map.set_position(-12.1235023, -77.02833272)
        self.map.set_zoom(15)
        
        
    def route(self):
        if self.entDesde.get() and self.entHacia.get():
            URL = "http://www.mapquestapi.com/directions/v2/route"
            payload = {'key': self.API_KEY, 
                      'from': self.entDesde.get(), 
                      'to': self.entHacia.get(), 
                      'unit': 'k',
                      'locale': 'es_MX'}
            
            r = requests.get(URL, params=payload)
            
            if r.status_code == requests.codes.ok:
                data = r.json()
                self.map.delete_all_path()
                self.map.delete_all_marker()
                self.map.fit_bounding_box((data['route']['boundingBox']['ul']['lat'],
                                          data['route']['boundingBox']['ul']['lng']), 
                                          (data['route']['boundingBox']['lr']['lat'],
                                          data['route']['boundingBox']['lr']['lng']))
                
                positions = []
                for item in data['route']['legs'][0]['maneuvers']: 
                    positions.append((item['startPoint']['lat'], item['startPoint']['lng']))
                else:
                    positions.append((data['route']['locations'][1]['latLng']['lat'], 
                                     data['route']['locations'][1]['latLng']['lng']))
        
            market_ini = self.map.set_marker(positions[0][0], positions[0][1], text=self.entDesde.get(), marker_color_circle='green')
            market_fin = self.map.set_marker(positions[-1][0], positions[-1][1], text=self.entHacia.get(), marker_color_circle='blue')
            path = self.map.set_path(positions, width=2)
        
        
App().mainloop()

## API Definidas por el usuario
Se puede definir una API en un servidor utilizando un Web Framework para crear servicios web en Python. De las varias opciones que existen, en esta ocasión utilizaremos el Frameword `flask`.

_Nota: En esta sección todos los códigos estan definidos en celdas tipo "Raw" para evitar su ejecución ya que los scripts de flask deben de ejcutarse en un terminal._

In [ ]:
!pip install flask

`flask` es un Framework que permite construir un servicio Web completo utilizando Python como Backend y gestión de base de datos (utilizando el ORM SQLAlchemy), así como herramientas adicionales para crear APIs. El estudio de `flask` esta fuera del alcance de este curso, sin embargo podemos conocer un poco de como crear una API utilizando `flask` y habilitando un par de métodos HTTP: `GET` y `POST`.

El código base de Flask es el siguiente:

    from flask import Flask, request, jsonify

    app = Flask(__name__)

    @app.route("/")
    def home():
        return "<h1>Hola mundo!</h1>"

    if __name__ == "__main__":
        app.run(debug=True)

Este código levanta un servidor Web en el equipo (en la dirección 172.0.0.1:5000 o localhost:5000) por defecto. En parametro `debug=True` permite que el servicio se actualice de forma automática cuando se guardan las modificaciones hechas en el código y mostrar mensaje de error o información sobre la ejecución del servicio. El decorador `@app.route` permite definir las rutas del URL que serán atendidas por el Webserver de Flask. Cuando ingrese un requerimiento a la ruta definida con el doecorador, se ejecuta la función decorada (en este caso, retorna un `str` que puede contener código HTML).

la función `request` de flask permite atender los requerimientos entrantes al servidor, y la función `jsonify` permite crear una respuesta JSON a partir de un diccionario. Con estos dos elementos, podemos construir un código que atienda a un requerimiento HTTP GET hacia el servidor.

    from flask import Flask, request, jsonify

    app = Flask(__name__)

    @app.route("/")
    def home():
        return "<h1>Hola mundo!</h1>"

    # 127.0.0.1:5000/get_user/12345
    @app.route("/get_user/<user_id>")
    def get_user_data(user_id):
        user_data = {'user_id': user_id, 
                     'name': "Elvio", 
                     'apellido': 'Lado'}

        # 127.0.0.1/get_user/12345?extra='extra'
        extra = request.args.get("extra")
        if extra:
            user_data['extra'] = extra

        return jsonify(user_data), 200


    if __name__ == "__main__":
        app.run(debug=True)

En el código anterior, la ruta `localhost:500/get_user/<user_id>` es atendida por la función `get_user_data` (por defecto el método es HTTP GET) que define un diccionario de respuesta y es retornado como una respuesta tipo JSON con `jsonify`, asi como el valor `200` que es la respuesta HTTP para una repuesta exitosa. Así también se tiene que la instrucción `request.args.get("query_field)` extrae del URL algun elemento de consulta del URL que este etiquetado como `query_field`. Estos elementos estan luego del caracter `?` en el URL de la forma `localhost:5000/get_user/12345?extra='extra_value`. En el ejemplo, se esta agregando este parametro de entrada al JSON de respuesta.

El otro método que se puede implementar es POST que permite insertar valores que llegaran en forma de un JSON. Para esto será neceario que el requerimiento incluya un JSON. Para recibir este bloque se define en el decorador que el método a atender sera `POST` y con la instrucción `requests.get_json` obtendremos el JSON que ha sido remitido en el `body` del requerimiento entrante (la respuesta 201 significa que se realizaron los cambios correctamente, ya que la intención de un POST requests es que se hagan cambios en una base de datos, por ejemplo).

    from flask import Flask, request, jsonify

    app = Flask(__name__)

    @app.route("/")
    def home():
        return "<h1>Hola mundo!</h1>"

    # 127.0.0.1:5000/get_user/12345
    @app.route("/get_user/<user_id>")
    def get_user_data(user_id):
        user_data = {'user_id': user_id, 
                     'name': "Elvio", 
                     'apellido': 'Lado'}

        # 127.0.0.1/get_user/12345?extra='extra'
        extra = request.args.get("extra")
        if extra:
            user_data['extra'] = extra

        return jsonify(user_data), 200


    @app.route("/set_user", methods=['POST'])
    def set_user():
        data = request.get_json()

        return jsonify(data), 201


    if __name__ == "__main__":
        app.run(debug=True)

Para probar el método POST del API, se puede ejecutar el siguiente script que envía un JSON en el requerimento.

    # Testing POST method
    import requests
    import json

    URL = "http://127.0.0.1:5000/set_user"
    data = {'user_id': 9999, 
            'nombre': 'Dina', 
            'apellido': 'Mita'}

    r = requests.post(URL, json=data)

    print(json.dumps(r.json(), indent=4))